In [1]:
!pip install duckdb
!pip install jupysql
!pip install duckdb-engine


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

Para cargar el archivo `.csv` a una tabla:

In [3]:
%%sql
create or replace table competencia_03 as 
select 
    *
from read_csv_auto('C:/Users/epugnalo/OneDrive - Telefonica/Documents/Data Mining UBA/2C - DM en Economica y Finanzas/datasets/competencia_03_crudo.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [4]:
%%sql
select *
from competencia_03
limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,0.0,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.0,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,0.0,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,0.0,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66
5,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,0.0,1,0.00,-5791.17,0.00,7827,12539.39,6,0,797.64
6,29188490,201901,1,0,1,43,281,-520.87,6116.69,2138.32,...,0.0,8,9172.86,-29810.71,0.00,7568,73132.65,41,0,9172.86
7,29188883,201901,1,0,1,64,296,64862.09,459723.16,988.28,...,0.0,8,0.00,-1332.76,0.00,7504,1102.50,1,0,58.65
8,29189950,201901,1,0,1,56,240,13535.41,41250.39,7158.16,...,0.0,1,0.00,-20419.25,66.25,3717,27701.56,19,0,1137.81
9,29190599,201901,1,0,1,59,296,2244.16,24670.53,113.76,...,0.0,22,8727.12,-16718.98,37.05,7137,1388.73,2,0,1219.92


In [5]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [6]:
%%sql

create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_03
), clientes as (
    select distinct numero_de_cliente from competencia_03
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0 --si en el mes no hay una foto para el cliente, devuelve 0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE WHEN foto_mes >=202108 THEN '' 
            WHEN mes_1 = 0 THEN 'BAJA+1'
            WHEN mes_2 = 0 THEN 'BAJA+2'
            ELSE 'CONTINUA' END as clase_ternaria

    from todo t
    left join competencia_03 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
  , mes_1
  , mes_2
from clase_ternaria where mes_0 = 1 
;


alter table competencia_03 add column clase_ternaria VARCHAR(10)
;

update competencia_03
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_03.numero_de_cliente = targets.numero_de_cliente and competencia_03.foto_mes = targets.foto_mes;
;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [11]:
%%sql

SELECT foto_mes, COUNT(*) cant, SUM(CASE WHEN clase_ternaria='BAJA+2' THEN 1 ELSE 0 END) cant_baja_mas_2,  ROUND(AVG(CASE WHEN clase_ternaria='BAJA+2' THEN 1 ELSE 0 END)*100,1) prop_baja_mas_2
FROM competencia_03
GROUP BY foto_mes


,foto_mes,cant,cant_baja_mas_2,prop_baja_mas_2
0,201901,124864,732.0,0.6
1,201902,125898,698.0,0.6
2,201903,126536,747.0,0.6
3,201904,127059,529.0,0.4
4,201905,127683,690.0,0.5
5,201906,129181,595.0,0.5
6,201907,130551,638.0,0.5
7,201908,132615,581.0,0.4
8,201909,134267,556.0,0.4
9,201910,136515,653.0,0.5


## Feature Engineering IntraMes

In [8]:
%%sql

CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);
CREATE MACRO resta_sin_null(a, b) AS ifnull(a, 0) - ifnull(b, 0);
CREATE MACRO DivisionPorCero(a, b) AS  CASE WHEN b = 0 THEN null ELSE ifnull(a,0) / b  END;

,Success


In [13]:
%%sql

create or replace table competencia_03_i1 as

WITH Feature_enginering as(

select *, 

    -- VISA + MASTERCARD  (MAXIMOS Y MINIMOS)

    greatest(Master_delinquency, Visa_delinquency) as tc_delinquency,
    greatest(Master_status, Visa_status) as tc_status_mayor,
    least(Master_status, Visa_status) as tc_status_menor,
    greatest(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as tc_mfinanciacion_mayor,
    least(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as tc_mfinanciacion_menor,    
    greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor,
    least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor,  
    greatest(Master_Finiciomora, Visa_Finiciomora) as tc_Finiciomora_mayor ,	
    least(Master_Finiciomora, Visa_Finiciomora) as tc_Finiciomora_menor ,
    greatest(Master_mlimitecompra, Visa_mlimitecompra) as tc_mlimitecompra_mayor ,	
    least(Master_mlimitecompra, Visa_mlimitecompra) as tc_mlimitecompra_menor ,
    greatest(Master_fultimo_cierre, Visa_fultimo_cierre) as tc_fultimo_cierre_mayor ,	
    least(Master_fultimo_cierre, Visa_fultimo_cierre) as tc_fultimo_cierre_menor ,
     greatest(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_mayor ,	
    least(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_menor ,
    greatest(Master_mpagominimo, Visa_mpagominimo) as tc_mpagominimo_mayor ,	
    least(Master_mpagominimo, Visa_mpagominimo) as tc_mpagominimo_menor,   

    -- VISA + MASTERCARD (SUMAS)
    suma_sin_null(Visa_mfinanciacion_limite, Master_mfinanciacion_limite) AS tc_mfinanciacion_limite_suma,
    suma_sin_null(Visa_msaldototal, Master_msaldototal) AS tc_msaldototal_suma,
    suma_sin_null(Visa_msaldopesos, Master_msaldopesos) AS tc_msaldopesos_suma,
    suma_sin_null(Visa_msaldodolares, Master_msaldodolares) AS tc_msaldodolares_suma,
    suma_sin_null(Visa_mconsumospesos, Master_mconsumospesos) AS tc_mconsumospesos_suma,
    suma_sin_null(Visa_mconsumosdolares, Master_mconsumosdolares) AS tc_mconsumosdolares_suma,
    suma_sin_null(Visa_mlimitecompra, Master_mlimitecompra) AS tc_mlimitecompra_suma,
    suma_sin_null(Visa_madelantopesos, Master_madelantopesos) AS tc_madelantopesos_suma,
    suma_sin_null(Visa_madelantodolares, Master_madelantodolares) AS tc_madelantodolares_suma,
    suma_sin_null(Visa_mpagado, Master_mpagado) AS tc_mpagado_suma,
    suma_sin_null(Visa_mpagospesos, Master_mpagospesos) AS tc_mpagospesos_suma,
    suma_sin_null(Visa_mpagosdolares, Master_mpagosdolares) AS tc_mpagosdolares_suma,
    suma_sin_null(Visa_mconsumototal, Master_mconsumototal) AS tc_mconsumototal_suma,
    suma_sin_null(Visa_cconsumos, Master_cconsumos) AS tc_cconsumos_suma,
    suma_sin_null(Visa_cadelantosefectivo, Master_cadelantosefectivo) AS tc_cadelantosefectivo_suma,
    suma_sin_null(Visa_mpagominimo, Master_mpagominimo) AS tc_mpagominimo_suma,


    -- OTRAS SUMAS y RESTAS
    suma_sin_null(mpasivos_margen, mactivos_margen) AS m_margen_suma,
    suma_sin_null(ccuenta_corriente, ccaja_ahorro) AS c_ccorrientes_cahorro_suma,
    suma_sin_null(mcuenta_corriente, mcuenta_corriente_adicional) AS m_ccorrientes_suma,
    suma_sin_null(mcaja_ahorro, mcaja_ahorro_adicional) AS m_cahorro_suma,
    suma_sin_null(ctarjeta_visa, ctarjeta_master) AS c_tarjetas_suma,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) AS  c_tarjetas_transacciones_suma,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) AS  mtarjetas_consumo_suma,

    suma_sin_null(cprestamos_prendarios, suma_sin_null(cprestamos_personales, cprestamos_hipotecarios)) AS  cprestamos_suma,
    suma_sin_null(mprestamos_personales, suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios)) AS  mprestamos_suma,

    suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_dolares) AS mplazo_fijo_suma,
    suma_sin_null(minversion1_pesos, suma_sin_null(minversion1_dolares, minversion2)) AS  minversion_suma,
    suma_sin_null(cseguro_vida,suma_sin_null(cseguro_auto, suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales))) AS  cseguro_suma,

    suma_sin_null(mpayroll, mpayroll2) AS mpayroll_suma,
    suma_sin_null(ccuenta_debitos_automaticos, suma_sin_null(ctarjeta_visa_debitos_automaticos, ctarjeta_master_debitos_automaticos)) AS  c_debitos_automaticos_suma,
    suma_sin_null(mcuenta_debitos_automaticos, suma_sin_null(mttarjeta_visa_debitos_automaticos, mttarjeta_master_debitos_automaticos)) AS  m_debitos_automaticos_suma,

    suma_sin_null(mcajeros_propios_descuentos, suma_sin_null(mtarjeta_visa_descuentos, mtarjeta_master_descuentos)) AS  m_descuentos_suma,
    suma_sin_null(mcomisiones_mantenimiento, mcomisiones_otras) AS mcomisiones_suma,
   
    suma_sin_null(ctransferencias_recibidas, ctransferencias_emitidas) AS ctransferencias_suma,
    resta_sin_null(ctransferencias_recibidas, ctransferencias_emitidas) AS ctransferencias_resta,
    resta_sin_null(mtransferencias_recibidas, mtransferencias_emitidas) AS mtransferencias_resta,

    suma_sin_null(mtransferencias_emitidas, mextraccion_autoservicio) AS m_extracciones_transf_suma,
    mpayroll+ mpayroll2+mcajeros_propios_descuentos+suma_sin_null(mtarjeta_visa_descuentos,mtarjeta_master_descuentos)+mtransferencias_recibidas+mcheques_depositados AS m_movimientos_positivos_suma,
    mautoservicio+mtarjeta_visa_consumo+mtarjeta_master_consumo+mcuenta_debitos_automaticos+mttarjeta_visa_debitos_automaticos+mttarjeta_master_debitos_automaticos+mpagodeservicios+mpagomiscuentas+mcomisiones_mantenimiento+mcomisiones_otras+mtransferencias_emitidas+mextraccion_autoservicio+mcheques_emitidos AS m_movimientos_negativos_suma

 from competencia_03

)

select *,
    DivisionPorCero(Visa_mconsumototal, Visa_mlimitecompra)*100 Visa_consumo_div_limite,
    DivisionPorCero(Master_mconsumototal, Master_mlimitecompra)*100 Master_consumo_div_limite,
    DivisionPorCero(tc_mconsumototal_suma, tc_mlimitecompra_suma)*100 tc_consumo_div_limite,
    DivisionPorCero(tc_mconsumototal_suma, tc_msaldototal_suma)*100 tc_consumo_div_saldo,
    DivisionPorCero(tc_mpagado_suma, tc_msaldototal_suma)*100 tc_pagado_div_saldo,

    DivisionPorCero(mprestamos_suma, mcuentas_saldo)*100 prestamos_saldo_div,    
    DivisionPorCero(m_descuentos_suma, mcuentas_saldo)*100 descuentos_saldo_div,    
    DivisionPorCero(mcomisiones_suma, mcuentas_saldo)*100 comisiones_saldo_div, 

    DivisionPorCero(m_extracciones_transf_suma, mcuentas_saldo)*100 extracciones_saldo_div, 
    DivisionPorCero(m_movimientos_negativos_suma, mcuentas_saldo)*100 mnegativos_saldo_div, 
    DivisionPorCero(m_movimientos_negativos_suma, m_movimientos_positivos_suma)*100 mpositivos_negativos_div
FROM Feature_enginering

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [14]:
%%sql

DROP TABLE competencia_03;

,Success


## Guardamos CSV

Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [15]:
%%sql
copy competencia_03_i1 to 'C:/Users/epugnalo/OneDrive - Telefonica/Documents/Data Mining UBA/2C - DM en Economica y Finanzas/datasets/competencia_03_v2.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
